## Implementation of A* algorithm for solving the puzzle 8 problem

This implementation is divided in two classes: Node and Puzzle

Node represents each of the positions the puzzle goes through until the end. It's in charge of generating a list with all the child nodes (in this case all the possible moves), making the moves, and comparing the states of 2 different nodes

In [0]:
""" class Node definition """


class Nodo:
    def __init__(self, data, level, idval, parent_node, cval, hval, fval):
        self.data = data
        self.level = level
        self.fval = fval
        self.hval = hval
        self.idval = idval
        self.cval = cval
        self.blank = self.encontrarHueco()
        self.parent_node = parent_node
        self.action_message = ''

    ''' Generate child nodes '''
    def crearHijo(self):
        x, y = self.blank[0], self.blank[1]
        """ val_list stores all the possible moves for a given cell """
        val_list = [[x, y - 1, 'Derecha'], [x, y + 1, 'Izquierda'], [x - 1, y, 'Abajo'], [x + 1, y, 'Arriba']]
        hijos = []
        idHijo = 1
        for i in val_list:
            hijo = self.mover(x, y, int(i[0]), int(i[1]))
            if hijo is not None:
                nodoHijo = Nodo(hijo, self.level + 1, self.idval + idHijo, self, self.cval + 1, 0, 0)
                nodoHijo.action_message = 'Mover ' + str(self.data[int(i[0])][int(i[1])]) + ' hacia ' + i[2]
                hijos.append(nodoHijo)
                idHijo += 1
        return hijos

    """ Move '_' cell from x1,y1 to x2,y2"""
    def mover(self, x1, y1, x2, y2):
        # Se comprueba si las coordenadas objetivos están dentro del rango de la matriz
        if x2 >= 0 and x2 < len(self.data) and y2 >= 0 and y2 < len(self.data):
            temp_puz = []
            temp_puz = self.copiar()
            temp = temp_puz[x2][y2]
            temp_puz[x2][y2] = temp_puz[x1][y1]
            temp_puz[x1][y1] = temp
            return temp_puz
        else:
            return None

    '''Make a copy of the current node'''
    def copiar(self):
        temp = []
        for i in self.data:
            t = []
            for j in i:
                t.append(j)
            temp.append(t)
        return temp

    def encontrarHueco(self):
        """ Finds and returns the position of the '_' cell"""
        for i in range(0, len(self.data)):
            for j in range(0, len(self.data)):
                if self.data[i][j] == '_':
                    return [i, j]

    def imprimirNodo(self):
        # Escribe información básica del nodo
        print('IdNodo:', str(self.idval), ',Profundidad:', str(self.level), ', Coste:', str(self.cval), ', Heuristica:',
              str(self.hval), ', f =', str(self.fval))
        # Escribe el estado que representa el nodo
        for i in self.data:
            for j in i:
                print(j, end=" ")
            print("")
        print(self.action_message)

    def comparar_estados(self, estado):
        for i in range(0, len(self.data)):
            for j in range(0, len(self.data)):
                if self.data[i][j] != estado[i][j]:
                    return False
        return True

The Puzzle class represents all the states the program goes through until solving the problem. It does the f-score, h-score and g-score calculation for the heuristics and finally implements the algorithm with the given functionality

In [0]:
""" Puzzle class definition """

class Puzzle:
    def __init__(self, size):
        """ Initialize the Puzle """
        self.n = size
        self.expandidos = []
        self.visitados = []

    def accept(self):
        puz = []
        for i in range(0, self.n):
            temp = input().split(" ")
            puz.append(temp)
        return puz

    def f(self, start, goal):
        """ Calculate the heuristic function value f(x) = h(x) + g(x) """
        return self.h(start.data, goal) + start.level

    def h(self, start, goal):
        """ h-score is the difference between the current node and the final node """
        distancia = 0
        for i in range(0, self.n):
            for j in range(0, self.n):
                if start[i][j] != goal[i][j] and start[i][j] != '_':
                    distancia += 1
        return distancia
    
    def explorar_nodo(self, nodo_nuevo):
        explorar = True
        continuar = True
        pos = 0
        # Checks if the current node has been visited
        while continuar and pos < len(self.visitados):
            nodo = self.visitados[pos]
            if nodo.comparar_estados(nodo_nuevo.data):
              if nodo.fval < nodo_nuevo.fval:
                explorar = False
                continuar = False
              
              elif nodo.fval == nodo_nuevo.fval and (nodo.hval <= nodo_nuevo.hval or nodo.cval <= nodo_nuevo.cval):
                explorar = False
                continuar = False
            pos += 1

        # Check if the f-value is lower than the previous node
        pos = 0   
        while continuar and pos < len(self.expandidos):
            nodo = self.expandidos[pos]
            if nodo.comparar_estados(nodo_nuevo.data):
              if nodo.fval < nodo_nuevo.fval:
                  explorar = False
                  continuar = False
              elif nodo.fval == nodo_nuevo.fval and (nodo.hval <= nodo_nuevo.hval or nodo.cval <= nodo_nuevo.cval):
                  explorar = False
                  continuar = False
              elif nodo.fval == nodo_nuevo.fval:
                  continuar = False
                  del self.expandidos[pos]
            pos += 1

        return explorar

    def process(self):
        print("Enter the initial state of the puzzle \n")
        iniciar = self.accept()
        print("Enter the result state \n")
        objetivo = self.accept()
        iniciar = Nodo(iniciar, 0, 0, None, 0, 0, 0)
        iniciar.fval = self.f(iniciar, objetivo)
        iniciar.hval = self.h(iniciar.data, objetivo)
        """ Add the start node to the open list """
        self.expandidos.append(iniciar)
        print("\n\n")
        solucion = []
        buscar = True
        while buscar and len(self.expandidos) > 0:
            cur = self.expandidos[0]
            """ when the hval is 0 it means that we have found are result state """
            if (cur.hval == 0):
                solucion.append(cur)
                # generate the path to the result state
                while cur.parent_node != None:
                    solucion.insert(0, cur.parent_node)
                    cur = cur.parent_node
                buscar = False
            else:
                # Explore the child nodes
                for i in cur.crearHijo():
                    i.fval = self.f(i, objetivo)
                    i.hval = self.h(i.data, objetivo)
                    # Check if the child node is in the way to the result and expand
                    if self.explorar_nodo(i):
                      self.expandidos.append(i)
                                      
                self.visitados.append(cur)
                del self.expandidos[0]
                """ Sort the nodes by f-value """
                self.expandidos.sort(key=lambda x: x.fval, reverse=False)

        for node in solucion:
            node.imprimirNodo()
            print('')


In [0]:
"""Play the game"""
puz = Puzzle(3)
puz.process()

Introducir el estado inicial del puzzle 

2 8 3
1 6 4
7 _ 5
Introducir el estado objetivo 

1 2 3
8 _ 4
7 6 5



Expandidos: 3  Visitados: 1
Expandidos: 5  Visitados: 2
Expandidos: 6  Visitados: 3
Expandidos: 7  Visitados: 4
Expandidos: 7  Visitados: 5
Expandidos: 8  Visitados: 6
IdNodo: 0 ,Profundidad: 0 , Coste: 0 , Heuristica: 4 , f = 4
2 8 3 
1 6 4 
7 _ 5 


IdNodo: 3 ,Profundidad: 1 , Coste: 1 , Heuristica: 3 , f = 4
2 8 3 
1 _ 4 
7 6 5 
Mover 6 hacia Abajo

IdNodo: 6 ,Profundidad: 2 , Coste: 2 , Heuristica: 3 , f = 5
2 _ 3 
1 8 4 
7 6 5 
Mover 8 hacia Abajo

IdNodo: 7 ,Profundidad: 3 , Coste: 3 , Heuristica: 2 , f = 5
_ 2 3 
1 8 4 
7 6 5 
Mover 2 hacia Derecha

IdNodo: 9 ,Profundidad: 4 , Coste: 4 , Heuristica: 1 , f = 5
1 2 3 
_ 8 4 
7 6 5 
Mover 1 hacia Arriba

IdNodo: 10 ,Profundidad: 5 , Coste: 5 , Heuristica: 0 , f = 5
1 2 3 
8 _ 4 
7 6 5 
Mover 8 hacia Izquierda

